In [20]:
import xarray as xr
import rioxarray as rxr
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
def scpf_to_scls_by_pft(scpf_var, dataset):
    """function to reshape a fates multiplexed size and pft-indexed variable to one indexed by size class and pft
    first argument should be an xarray DataArray that has the FATES SCPF dimension
    second argument should be an xarray Dataset that has the FATES SCLS dimension 
    (possibly the dataset encompassing the dataarray being transformed)
    returns an Xarray DataArray with the size and pft dimensions disentangled"""
    n_scls = len(dataset.fates_levscls)
    return (scpf_var.rolling(fates_levscpf=n_scls, center=False)
            .construct("fates_levscls")
            .isel(fates_levscpf=slice(n_scls-1, None, n_scls))
            .rename({'fates_levscpf':'fates_levpft'})
            .assign_coords({'fates_levscls':dataset.fates_levscls}))

def scag_to_scls_by_age(scag_var, dataset):
    """function to reshape a fates multiplexed size and pft-indexed variable to one indexed by size class and pft
    first argument should be an xarray DataArray that has the FATES SCAG dimension
    second argument should be an xarray Dataset that has the FATES age dimension 
    (possibly the dataset encompassing the dataarray being transformed)
    returns an Xarray DataArray with the size and age dimensions disentangled"""
    n_age = len(dataset.fates_levage)
    return (scag_var.rolling(fates_levscag=n_age, center=False)
            .construct("fates_levage")
            .isel(fates_levscag=slice(n_age-1, None, n_age))
            .rename({'fates_levscag':'fates_levpft'})
            .assign_coords({'fates_levage':dataset.fates_levage}))


def monthly_to_annual(array):
    mon_day  = xr.DataArray(np.array([31.,28.,31.,30.,31.,30.,31.,31.,30.,31.,30.,31.]), dims=['month'])
    mon_wgt  = mon_day/mon_day.sum()
    return (array.rolling(time=12, center=False) # rolling
            .construct("month") # construct the array
            .isel(time=slice(11, None, 12)) # slice so that the first element is [1..12], second is [13..24]
            .dot(mon_wgt, dims=["month"]))

def monthly_to_month_by_year(array):
    return (array.rolling(time=12, center=False) # rolling
            .construct("month") # construct the array
            .isel(time=slice(11, None, 12)).rename({'time':'year'}))





In [3]:
data_path = "C:/Users/345578/Documents/GitHub/Transfer_HPC/FATES_RUNS/Examplestuntridge.nc"

Fates_xr  = xr.open_dataset(data_path)  
# View xarray object
Fates_xr

<xarray.Dataset>
Dimensions:                          (fates_levage: 7, fates_levagefuel: 42, fates_levagepft: 14, fates_levcacls: 2, fates_levcan: 2, fates_levcapf: 4, fates_levcnlf: 60, fates_levcnlfpf: 120, fates_levcwdsc: 4, fates_levelage: 7, fates_levelcwd: 4, fates_levelem: 1, fates_levelpft: 2, fates_levfuel: 6, fates_levheight: 6, fates_levpft: 2, fates_levscag: 91, fates_levscagpf: 182, fates_levscls: 13, fates_levscpf: 26, hist_interval: 2, levdcmp: 15, levgrnd: 15, levlak: 10, lndgrid: 1, ltype: 9, natpft: 17, time: 115)
Coordinates: (12/13)
  * levgrnd                          (levgrnd) float32 0.007101 ... 35.18
  * levlak                           (levlak) float32 0.05 0.6 ... 34.33 44.78
  * levdcmp                          (levdcmp) float32 0.007101 ... 35.18
  * fates_levscls                    (fates_levscls) float32 0.0 5.0 ... 100.0
  * fates_levcacls                   (fates_levcacls) float32 0.0 5.0
  * fates_levage                     (fates_levage) float32 0.0 1.0 ... 50.0
    ...                               ...
  * fates_levfuel                    (fates_levfuel) int32 1 2 3 4 5 6
  * fates_levcwdsc                   (fates_levcwdsc) int32 1 2 3 4
  * fates_levcan                     (fates_levcan) int32 1 2
  * fates_levheight                  (fates_levheight) float32 0.0 0.1 ... 10.0
  * fates_levelem                    (fates_levelem) int32 1
  * time                             (time) object 1931-02-01 00:00:00 ... 19...
Dimensions without coordinates: fates_levagefuel, fates_levagepft, fates_levcapf, fates_levcnlf, fates_levcnlfpf, fates_levelage, fates_levelcwd, fates_levelpft, fates_levscag, fates_levscagpf, fates_levscpf, hist_interval, lndgrid, ltype, natpft
Data variables: (12/571)
    fates_scmap_levscag              (fates_levscag) int32 ...
    fates_agmap_levscag              (fates_levscag) int32 ...
    fates_pftmap_levscpf             (fates_levscpf) int32 ...
    fates_scmap_levscpf              (fates_levscpf) int32 ...
    fates_pftmap_levcapf             (fates_levcapf) int32 ...
    fates_camap_levcapf              (fates_levcapf) int32 ...
    ...                               ...
    WASTEHEAT                        (time, lndgrid) float32 ...
    WIND                             (time, lndgrid) float32 ...
    ZBOT                             (time, lndgrid) float32 ...
    ZSTAR_BY_AGE                     (time, fates_levage, lndgrid) float32 ...
    ZWT                              (time, lndgrid) float32 ...
    ZWT_PERCH                        (time, lndgrid) float32 ...
Attributes: (12/28)
    title:                                     ELM History file information
    source:                                    E3SM Land Model
    source_id:                                 ead8f1ada2
    product:                                   model-output
    realm:                                     land
    case:                                      SR.IELMFATES.badger.intel.Cead...
    ...                                        ...
    ltype_deep_lake:                           5
    ltype_wetland:                             6
    ltype_urban_tbd:                           7
    ltype_urban_hd:                            8
    ltype_urban_md:                            9
    nco_openmp_thread_number:                  1

In [5]:
tave_start = 40*12
tave_stop  = 60*12

minlat = Fates_xr.variables['lat'].min()
maxlat = Fates_xr.variables['lat'].max()
minlon = Fates_xr.variables['lon'].min()
maxlon = Fates_xr.variables['lon'].max()


nscls_bins = len(Fates_xr.variables['fates_levscls'])
npfts = len(Fates_xr.variables['fates_levpft'])
nage_bins = len(Fates_xr.variables['fates_levage'])

JM_fates = len(Fates_xr.variables['lat'])
IM_fates = len(Fates_xr.variables['lon'])

In [18]:
Fates_xr.variables


Frozen({'levgrnd': <xarray.IndexVariable 'levgrnd' (levgrnd: 15)>
array([7.100635e-03, 2.792500e-02, 6.225858e-02, 1.188651e-01, 2.121934e-01,
       3.660658e-01, 6.197585e-01, 1.038027e+00, 1.727635e+00, 2.864607e+00,
       4.739157e+00, 7.829766e+00, 1.292532e+01, 2.132647e+01, 3.517762e+01],
      dtype=float32)
Attributes:
    long_name:  coordinate soil levels
    units:      m, 'levlak': <xarray.IndexVariable 'levlak' (levlak: 10)>
array([ 0.05 ,  0.6  ,  2.1  ,  4.6  ,  8.1  , 12.6  , 18.6  , 25.6  , 34.325,
       44.775], dtype=float32)
Attributes:
    long_name:  coordinate lake levels
    units:      m, 'levdcmp': <xarray.IndexVariable 'levdcmp' (levdcmp: 15)>
array([7.100635e-03, 2.792500e-02, 6.225858e-02, 1.188651e-01, 2.121934e-01,
       3.660658e-01, 6.197585e-01, 1.038027e+00, 1.727635e+00, 2.864607e+00,
       4.739157e+00, 7.829766e+00, 1.292532e+01, 2.132647e+01, 3.517762e+01],
      dtype=float32)
Attributes:
    long_name:  coordinate soil levels
    units:    

In [22]:
Fates_xr.coords

Coordinates: (12/13)
  * levgrnd          (levgrnd) float32 0.007101 0.02792 0.06226 ... 21.33 35.18
  * levlak           (levlak) float32 0.05 0.6 2.1 4.6 ... 18.6 25.6 34.33 44.78
  * levdcmp          (levdcmp) float32 0.007101 0.02792 0.06226 ... 21.33 35.18
  * fates_levscls    (fates_levscls) float32 0.0 5.0 10.0 ... 80.0 90.0 100.0
  * fates_levcacls   (fates_levcacls) float32 0.0 5.0
  * fates_levage     (fates_levage) float32 0.0 1.0 2.0 5.0 10.0 20.0 50.0
    ...               ...
  * fates_levfuel    (fates_levfuel) int32 1 2 3 4 5 6
  * fates_levcwdsc   (fates_levcwdsc) int32 1 2 3 4
  * fates_levcan     (fates_levcan) int32 1 2
  * fates_levheight  (fates_levheight) float32 0.0 0.1 0.3 1.0 3.0 10.0
  * fates_levelem    (fates_levelem) int32 1
  * time             (time) object 1931-02-01 00:00:00 ... 1940-08-01 00:00:00

In [24]:
list(Fates_xr.keys())

['fates_scmap_levscag',
 'fates_agmap_levscag',
 'fates_pftmap_levscpf',
 'fates_scmap_levscpf',
 'fates_pftmap_levcapf',
 'fates_camap_levcapf',
 'fates_canmap_levcnlf',
 'fates_lfmap_levcnlf',
 'fates_canmap_levcnlfpf',
 'fates_lfmap_levcnlfpf',
 'fates_pftmap_levcnlfpf',
 'fates_scmap_levscagpft',
 'fates_agmap_levscagpft',
 'fates_pftmap_levscagpft',
 'fates_pftmap_levagepft',
 'fates_agmap_levagepft',
 'fates_elmap_levelpft',
 'fates_pftmap_levelpft',
 'fates_elmap_levelcwd',
 'fates_cwdmap_levelcwd',
 'fates_elmap_levelage',
 'fates_agemap_levelage',
 'fates_agmap_levagefuel',
 'fates_fscmap_levagefuel',
 'mcdate',
 'mcsec',
 'mdcur',
 'mscur',
 'nstep',
 'time_bounds',
 'date_written',
 'time_written',
 'lon',
 'lat',
 'area',
 'topo',
 'landfrac',
 'landmask',
 'pftmask',
 'ZSOI',
 'DZSOI',
 'WATSAT',
 'SUCSAT',
 'BSW',
 'HKSAT',
 'ZLAKE',
 'DZLAKE',
 'ACTUAL_IMMOB',
 'ACTUAL_IMMOB_P',
 'ADSORBTION_P',
 'AGB',
 'AGB_SCLS',
 'AR',
 'AREA_BURNT_BY_PATCH_AGE',
 'AREA_PLANT',
 'ARE

In [25]:
xr.DataArray(Fates_xr, coords=[time, levage], dims=["time", "age"])


NameError: name 'time' is not defined